In [1]:
import json
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Load Model

In [3]:
# load file

# Modelling
with open('random_model.pkl', 'rb') as file_1:
    best_estimator = pickle.load(file_1)


# Pre-processing

with open('scaler.pkl', 'rb') as file_2:
    scaler = pickle.load(file_2)
    
with open('winsoriser.pkl','rb') as file_3:
    winsoriser = pickle.load(file_3)
    
# List Numeric & Category
with open('num_cols_sc.txt', 'r') as file_4:
    num_cols_scaled = json.load(file_4)

with open('num_cols_nsc.txt', 'r') as file_5:
    num_cols_non_scaled = json.load (file_5)
    

# 11. Model Inferencing

In [6]:
# load dataset
data_inferences=pd.read_csv('h8dsft_P1G3_iqbal.csv')

In [7]:
df_inferences = data_inferences.head(10) 

In [8]:
df_inferences

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,42.0,1,250,1,15,0,213000.0,1.3,136,0,0,65,1
1,46.0,0,168,1,17,1,271000.0,2.1,124,0,0,100,1
2,65.0,1,160,1,20,0,327000.0,2.7,116,0,0,8,1
3,53.0,1,91,0,20,1,418000.0,1.4,139,0,0,43,1
4,50.0,1,582,1,20,1,279000.0,1.0,134,0,0,186,0
5,70.0,1,125,0,25,1,237000.0,1.0,140,0,0,15,1
6,65.0,1,52,0,25,1,276000.0,1.3,137,0,0,16,0
7,70.0,0,161,0,25,0,244000.0,1.2,142,0,0,66,1
8,60.0,1,76,1,25,0,196000.0,2.5,132,0,0,77,1
9,59.0,1,280,1,25,1,302000.0,1.0,141,0,0,78,1


In [9]:
df_inferences_use = df_inferences.drop(['DEATH_EVENT'] ,axis=1)

# OUTLIER


In [10]:
df_inferences_out= winsoriser.transform(df_inferences_use)

## Spliting data before Scaling

In [11]:
df_inferences_sc=df_inferences_out[num_cols_scaled]
df_inferences_nsc=df_inferences_out[num_cols_non_scaled]

In [12]:
df_inferences_sc=scaler.transform(df_inferences_sc)

In [13]:
df_inferences_sc = pd.DataFrame(df_inferences_sc, columns=num_cols_scaled)

In [14]:
df_inferences_sc.reset_index(drop=True ,  inplace=True)
df_inferences_nsc.reset_index(drop=True  , inplace=True)

In [15]:
df_con = pd.concat([df_inferences_sc , df_inferences_nsc] ,axis=1)

# Modeling


In [17]:
y_predict_inferences = best_estimator.predict(df_con)

In [18]:
y_predict_inferences 

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1], dtype=int64)

In [19]:
df_y_predict_inferences = pd.DataFrame(y_predict_inferences, columns=['Prediction'])

In [20]:
df_inferences_final = pd.concat([df_inferences , df_y_predict_inferences] , axis=1)

In [21]:
df_inferences_final

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT,Prediction
0,42.0,1,250,1,15,0,213000.0,1.3,136,0,0,65,1,1
1,46.0,0,168,1,17,1,271000.0,2.1,124,0,0,100,1,1
2,65.0,1,160,1,20,0,327000.0,2.7,116,0,0,8,1,1
3,53.0,1,91,0,20,1,418000.0,1.4,139,0,0,43,1,1
4,50.0,1,582,1,20,1,279000.0,1.0,134,0,0,186,0,0
5,70.0,1,125,0,25,1,237000.0,1.0,140,0,0,15,1,1
6,65.0,1,52,0,25,1,276000.0,1.3,137,0,0,16,0,1
7,70.0,0,161,0,25,0,244000.0,1.2,142,0,0,66,1,1
8,60.0,1,76,1,25,0,196000.0,2.5,132,0,0,77,1,1
9,59.0,1,280,1,25,1,302000.0,1.0,141,0,0,78,1,1


berbandingan 10 data awal dan data prediction menunjukan hanya ada 1 prediction yang salah

# 12. Kesimpulan

Exploratory Data Analysis (EDA) : 

* Dari seluruh data jumlah pasien hidup lebih banyak ketimbang pasien meninggal dengan jumlah perbandingan 203 : 96

* Rentang umur pasien meninggal terbanyak diangka 70 tahun keatas

* Berdasarkan data lebih banyak orang yang tidak merokok tp meninggal ketimbang yang merokok .



Hasil Kesimpulan Model : 
* Model `RandomForest` tergolong overfit 
    * ROC-AUC Train :  0.9904200737534069 

    * ROC-AUC Test:  0.8318356867779204
    
* Nilai `Recall` cukup baik yaitu 0.736. Karena meminimalisir adanya kesalahan prediksi jika pasian yang ternyata meninggal terprediksi hidup.
* Nilai `ROC AUC` di `RandomForest` lebih baik ketimbang `AdaBoost`


Kelebihan : 
* nilai `recall` cukup baik untuk prediksi . karena dalam dunia kesehatan sangat riskan terhadap prediksi. Oleh karena itu lebih baik terdapat pasien yang prediksi bahwa dia mati tapi actualnya hidup, dari pada pasien tersebut terprediksi hidup tapi actual nya meninggal.

Kelemahan : 
* model masih tergolong overfit antara train dan test

